In [ ]:
import pandas as pd
import numpy as np
import implicit
from scipy.sparse import coo_matrix
from typing import List
from tqdm import tqdm
import gc
tqdm.pandas()
from rectools import Columns
from rectools.metrics.ranking import NDCG, MRR, MAP
from rectools.metrics.classification import Precision, Recall
from rectools.metrics.serendipity import Serendipity
from rectools.metrics.novelty import MeanInvUserFreq

In [ ]:
df = pd.read_csv("train_dataset.csv")

In [ ]:
df.shape

(50000000, 5)

In [ ]:
df.head()

,event_datetime,viewer_id,video_id,title,author_name
0,2024-03-04 22:34:36+03:00,f7cbc2c9243e442c914c37b6bca52248,1192676af5b55a3798f33336485b0de6,Сериал Дарреллы | The Durrells - 2 сезон 5 серия,Сериал Дарреллы | The Durrells
1,2024-03-04 09:25:38+03:00,12263c993f1c4feeab33d8fb4193c417,1192f0bbb8e7d1bfd7a732b1266a0d7f,Как разобрать коленвал снегохода буран или под...,Как отремонтировать и как ловить
2,2024-03-04 21:06:47+03:00,25fbc72a-b4b4-4c70-959e-010fbf88d82e,1192f0bbb8e7d1bfd7a732b1266a0d7f,Как разобрать коленвал снегохода буран или под...,Как отремонтировать и как ловить
3,2024-03-04 02:53:16+03:00,6ca51ef9-e673-4098-8f59-11e38eec62cc,1192f0bbb8e7d1bfd7a732b1266a0d7f,Как разобрать коленвал снегохода буран или под...,Как отремонтировать и как ловить
4,2024-03-04 04:38:35+03:00,a05532cf-92e7-4567-b50c-39572dcbf8e4,1192f0bbb8e7d1bfd7a732b1266a0d7f,Как разобрать коленвал снегохода буран или под...,Как отремонтировать и как ловить


#### 0. Разделение выборки на трейн и тест
##### Данные для тренировки и тестирования моделей рекомендательных систем важно делить по времени. Выбирается отсечка и все что до нее - это трейн, а все что после - тест

In [ ]:
# Отделить 1 день на тестирование - 16-03-2024
test = df.query("event_datetime >= '2024-03-16'")
df = df.query("event_datetime < '2024-03-16'")

#### 1. Alternating Least Squares (ALS)
##### Матричное разложение на представления пользователей и айтемов с попеременным обучением каждой из матриц представлений при фиксированной второй матрице (сначала обучаем представления пользователей, фиксируя представления айтемов, потом наоборот и тд)

In [ ]:
ALL_USERS = df['viewer_id'].unique().tolist()
ALL_ITEMS = df['video_id'].unique().tolist()

user_ids = dict(list(enumerate(ALL_USERS)))
item_ids = dict(list(enumerate(ALL_ITEMS)))

user_map = {u: uidx for uidx, u in user_ids.items()}
item_map = {i: iidx for iidx, i in item_ids.items()}

df['user_id'] = df['viewer_id'].map(user_map)
df['item_id'] = df['video_id'].map(item_map)

In [ ]:
row = df['user_id'].values
col = df['item_id'].values
data = np.ones(df.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<6097098x1038746 sparse matrix of type '<class 'numpy.float64'>'
	with 45849018 stored elements in COOrdinate format>

In [ ]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=200, iterations=10)
model.fit(coo_train)

/home/rkorst/Whispe_steve_demo/experiment/alex_klyuev/services/Speech2Text/recomendation_systems/venv_rec_sys/lib/python3.8/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 2.6561012268066406 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 41.3 s, sys: 340 ms, total: 41.7 s
Wall time: 41.7 s


In [ ]:
video_id_to_title = {}

for row in tqdm(df.itertuples()):
    video_id_to_title[row.video_id] = row.title
len(video_id_to_title)

45849018it [01:06, 691631.51it/s]


1038746

In [ ]:
def get_recs_by_als(model, video_id: str, num_candidates: int) -> List[str]:
    video_id_to_idx = {v: k for k, v in item_ids.items()}
    item_index = video_id_to_idx.get(video_id, None)
    if item_index or item_index == 0:
        # skip first recommendation because it finds the same item
        recommendations = model.similar_items(
            item_index, N=num_candidates, filter_items=[item_index]
        )[0]
    else:
        return []
    return list(map(lambda x: video_id_to_title[item_ids[x]], recommendations))

In [ ]:
video_id = "69bdad2a1ad8105c5bb8b1adc0c32bf8" # Человек-паук: Нет пути домой | Spider-Man: No Way Home (2021)
get_recs_by_als(model, video_id, 30)

['Человек-паук: Вдали от дома | Spider-Man: Far from Home (2019)',
 'Тор: Любовь и гром (фильм, 2022)',
 'Тор: Рагнарёк (фильм, 2017)',
 'Человек-паук: Возвращение домой | Spider-Man: Homecoming (2017)',
 'Тор | Thor (2011)',
 'Мстители: Финал (фильм, 2019)',
 'Первый мститель | Captain America: The First Avenger (2011)',
 'Тор: Рагнарёк | Thor: Ragnarok (2017)',
 'Чёрная Пантера (фильм, 2018)',
 'Красное уведомление 1 (фильм, 2021)',
 'Мстители: Война бесконечности | Avengers: Infinity War (2018)',
 'Мстители, 2012',
 'Капитан Марвел 2  | Captain Marvel 2 (2023)',
 'Тор 2: Царство тьмы | Thor: The Dark World (2013)',
 'Чёрная Пантера: Ваканда навеки | Black Panther: Wakanda Forever (2022)',
 'Человек-муравей и Оса: Квантомания (фильм, 2023)',
 'Мстители: Эра Альтрона | Avengers: Age of Ultron (2015)',
 'Первый мститель: Противостояние | Captain America: Civil War (2016)',
 'Шан-Чи и легенда десяти колец (фильм, 2021)',
 'Новый Человек-паук | The Amazing Spider-Man (2012)',
 'Мстители:

In [ ]:
video_id = "95c71e40cd1148b95d7a057fe3718866" # Гарри Поттер и Орден Феникса | Harry Potter and the Order of the Phoenix (2007)
get_recs_by_als(model, video_id, 30)

['Гарри Поттер и Кубок огня | Harry Potter and the Goblet of Fire (2005)',
 'Гарри Поттер и Принц-полукровка | Harry Potter and the Half-Blood Prince (2009)',
 'Гарри Поттер и Тайная комната | Harry Potter and the Chamber of Secrets (2002)',
 'Гарри Поттер и Кубок огня 2005',
 'Гарри Поттер и узник Азкабана| Harry Potter and the Prisoner of Azkaban (2004)',
 'Гарри Поттер и Проклятое Дитя - Первый Трейлер (2025) По Мотивам Книги _ Концепт-версия от Тизер ПРО',
 'Гарри Поттер и Принц-полукровка 2009',
 'Гарри Поттер и Дары Смерти: Часть 1. 2010',
 'Гарри Поттер и Дары смерти: Часть 1 | Harry Potter and the Deathly Hallows - Part 1 (2010)',
 'Гарри Поттер и Проклятое Дитя - Первый (2025)',
 'Гарри Поттер и узник Азкабана (фильм, 2004, 3 часть)',
 "Гарри Поттер и философский камень | Harry Potter and the Sorcerer's Stone (2001)",
 'Гарри Поттер и Кубок огня (фильм, 2005, 4 часть)',
 'Гарри Поттер и Орден Феникса (фильм, 2007, 5 часть)',
 'Гарри Поттер 20 лет спустя: Возвращение в Хогвартс

In [ ]:
video_id = "00256d60056a0251fb0e71464536bc50" # Смешарики, 2 серия
get_recs_by_als(model, video_id, 30)

['Смешарики, 1 серия',
 'Смешарики, 3 серия',
 'Смешарики, 6 серия',
 'Смешарики, 17 серия',
 'Смешарики 2D, 5 сезон, 35 серия',
 'Смешарики, 11 серия',
 'Смешарики 2D, 5 сезон, 52 серия',
 'Смешарики, 8 серия',
 'Смешарики, 23 серия',
 'Смешарики, 12 серия',
 'Смешарики. Клипы. Тик-так, 15 серия',
 'Смешарики 2D, 6 сезон, 4 серия. Жажда',
 'Смешарики, 21 серия',
 'Смешарики 2D, 6 сезон, 1 серия. Паранормальное приключение',
 'Монсики, Ролик Наушники',
 'Супер Зак, 2 сезон, 2 серия. Бежим! Рон-Молния!',
 'Фиксики, 90 серия',
 'Фиксики, 34 серия',
 'Смешарики 2D, 6 сезон, 23 серия. Окрошка',
 'Смешарики 2D, 6 сезон, 3 серия. Гусений',
 'Фиксики, 99 серия',
 'Смешарики, 7 серия',
 'Смешарики 2D, 5 сезон, 3 серия',
 'Смешарики, 4 серия',
 'Фиксики, 199 серия',
 'Мультик про Машинки - Экскаватор, Бульдозер, Бетономешалка и Самосвал. МанкиМульт',
 'Фиксики, 104 серия',
 'Школа Шишкиного Леса. Рукоделие. Игрушка-мобиль',
 'Зебра в клеточку -  28 серия. Новый год - Союзмультфильм HD',
 'Фикси

In [ ]:
video_id = "72b83e3fdefb9d5492b693a4c654ef10" # Спартак - Зенит. Обзор матча МИР РПЛ 20.08.2023
get_recs_by_als(model, video_id, 30)

['Сочи - Балтика. Обзор матча Мир РПЛ 23.07.2023',
 'ЦСКА - Динамо. Обзор матча МИР РПЛ 25.11.2023',
 'ПСО-13 - Пульсар | Золотой плей-офф| 1/8 финала',
 'ЦСКА - Оренбург. Обзор матча Мир РПЛ 02.03.2024',
 'Ростов - Крылья Советов. Обзор матча Мир РПЛ 01.03.2024',
 'Спартак - ЦСКА. Обзор матча МИР РПЛ 08.10.2023',
 'Крылья Советов - ЦСКА. Обзор матча Мир РПЛ 08.03.2024',
 'Краснодар - Рубин. Обзор матча Мир РПЛ 02.03.2024',
 'Ростов - Краснодар. Обзор матча Мир РПЛ 08.03.2024',
 'Ахмат - Урал. Обзор матча Мир РПЛ 02.03.2024',
 'Крылья Советов - Ростов. Обзор матча Мир РПЛ 05.08.2023',
 'Краснодар - Ростов. Обзор матча МИР РПЛ 07.10.2023',
 'Химки - Краснодар. Обзор матча FONBET Кубка России 12.03.2024',
 'Локомотив - Балтика. Обзор матча FONBET Кубка России 14.03.2024',
 'Динамо - Ахмат. Обзор матча Мир РПЛ 09.03.2024',
 'Зенит - Спартак. Обзор матча Мир РПЛ 02.03.2024',
 'Урал - Зенит. Обзор матча Мир РПЛ 09.03.2024',
 '«Что за спорт». Выпуск от 20.02.2024',
 'Рубин - Пари НН. Обзор м

In [ ]:
video_id = "52adf40199ec58315a17826835b0b03a" # Comedy Club: Управляющая компания | Карибидис, Батрутдинов
get_recs_by_als(model, video_id, 30)

['Comedy Club: Случай в самолёте | Карибидис, Батрутдинов',
 'Comedy Club: Барды | Харламов, Батрутдинов',
 'Comedy Club: Устройся на работу  | Марина Кравец, Демис Карибидис,  Костя Бутусов',
 'Comedy Club: Робот | Демис Карибидис, Тимур Батрутдинов, Гарик Харламов',
 'Comedy Club: Утро после корпоратива | Харламов, Карибидис, Батрутдинов, Аверин, Скороход',
 'Comedy Club: Романтический ужин | Карибидис, Кравец, Скороход, Темичева',
 'Comedy Club: Счастливый муж | Демис Карибидис, Марина Кравец',
 'Comedy Club: Развел девушку | Гарик Харламов, Костя Бутусов, Катя Шкуро',
 'Comedy Club: Cумасшедший начальник | Харламов, Батрутдинов, Бутусов, Шкуро, Шальнов',
 'Comedy Club: Писательница | Демис Карибидис, Марина Кравец',
 'Comedy Club: Новогодний корпоратив | Кравец, Карибидис, Батрутдинов',
 'Comedy Club: Женщина-абьюзер | Марина Кравец, Демис Карибидис, Тимур Батрутдинов, Костя Бутусов',
 'Comedy Club: Завещание | Гарик Харламов, Марина Федункив, Алексей Шальнов',
 'Comedy Club: Дотош

#### 2. Ассоциативные правила
##### Подсчет совстречаемостей айтемов вместе в сессиях пользователей и нормировка этих совстречаемостей на безотносительную встречаемость айтемов

In [ ]:
# preprocess date
df['date_short'] = df.event_datetime.progress_apply(lambda l: l.split(" ")[0])

100%|██████████| 45849018/45849018 [00:45<00:00, 1008677.92it/s]


In [ ]:
%%time
groupby = df.groupby(["user_id", "date_short"]).count()
users_active = groupby[groupby['item_id'] >= 2].index.unique()

CPU times: user 27.1 s, sys: 1.3 s, total: 28.4 s
Wall time: 28.4 s


In [ ]:
users_active = set(users_active)

In [ ]:
df['active'] = df.progress_apply(lambda l: True if (l.user_id, l.date_short) in users_active else False, axis=1)
df=df[df.active]

100%|██████████| 45849018/45849018 [10:10<00:00, 75050.20it/s]


In [ ]:
def calc_pairs(train):
    print("Get sessions")
    dt = train.groupby(['user_id', 'date_short'])['item_id'].agg(list).rename('pair').reset_index()
    df = train[['user_id', 'date_short', 'item_id']].merge(dt, on=['user_id', 'date_short'], how='left')

    del dt
    gc.collect()

    # Explode the rows vs list of articles
    print("Explode the rows vs list of articles")
    df = df[['item_id', 'pair']].explode(column='pair')
    gc.collect()

    print("Discard duplicates")
    df = df.loc[df['item_id'] != df['pair']].reset_index(drop=True)

    print("Count how many times each pair combination happens")
    df = df.groupby(['item_id', 'pair']).size().rename('count').reset_index()

    df['score'] = df.apply(lambda l: l['count'], axis=1)
    df.sort_values(by=['score'], ascending=False, inplace=True)
    gc.collect()
    return df

In [ ]:
%%time
rules = calc_pairs(df[['user_id', 'item_id', 'date_short']])

Get sessions
Explode the rows vs list of articles
Discard duplicates
Count how many times each pair combination happens
CPU times: user 17min, sys: 1min 3s, total: 18min 4s
Wall time: 18min 3s


In [ ]:
rules.to_csv("rules_3.csv", index=False)

In [ ]:
rules = pd.read_csv("rules_3.csv")

In [ ]:
def get_recs_by_i2i(rules, video_id: str, num_candidates: int) -> List[str]:
    video_id_to_idx = {v: k for k, v in item_ids.items()}
    item_index = video_id_to_idx.get(video_id, None)
    if item_index or item_index == 0:
        # skip first recommendation because it finds the same item
        recommendations = rules[rules.item_id==item_index].pair.to_list()[:num_candidates]
    else:
        return []
    return list(map(lambda x: video_id_to_title[item_ids[x]], recommendations))

In [ ]:
video_id = "69bdad2a1ad8105c5bb8b1adc0c32bf8" # Человек-паук: Нет пути домой | Spider-Man: No Way Home (2021)
get_recs_by_i2i(rules, video_id, 30)

['Человек-паук: Вдали от дома | Spider-Man: Far from Home (2019)',
 'Человек-паук: Возвращение домой | Spider-Man: Homecoming (2017)',
 'Человек-паук: Нет пути домой (фильм, 2021)',
 'Новый Человек-паук: Высокое напряжение | The Amazing Spider-Man 2 (2014)',
 'Доктор Стрэндж 2: В мультивселенной безумия (фильм, 2022)',
 'Новый Человек-паук | The Amazing Spider-Man (2012)',
 'Мстители: Война бесконечности (фильм, 2018)',
 'Человек-паук 3: Враг в отражении | Spider-Man 3 (2007)',
 'Веном 2 (фильм, 2021)',
 'Человек-паук: Через вселенные | Spider-Man: Into the Spider-Verse (2018)',
 'Человек Паук 4 - Официальный Трейлер (2025)',
 'Выжить в Самарканде, 1 сезон, 8 выпуск. ФИНАЛ',
 'Мстители: финал (фильм, 2019, 4 часть)',
 'Человек-паук 1 (фильм, 2002)',
 'Человек-паук: Через вселенные (мультфильм, 2018)',
 'Человек-паук: Паутина вселенных | Spider-Man: Across the Spider-Verse (2023)',
 'Тор: Любовь и гром (фильм, 2022)',
 'Мстители: Финал | Avengers: Endgame (2019)',
 'Железный человек 200

In [ ]:
video_id = "95c71e40cd1148b95d7a057fe3718866" # Гарри Поттер и Орден Феникса | Harry Potter and the Order of the Phoenix (2007)
get_recs_by_i2i(rules, video_id, 30)

['Гарри Поттер и Принц-полукровка | Harry Potter and the Half-Blood Prince (2009)',
 'Гарри Поттер и Кубок огня | Harry Potter and the Goblet of Fire (2005)',
 'Гарри Поттер и Принц-полукровка (фильм, 2009, 6 часть)',
 'Гарри Поттер и узник Азкабана| Harry Potter and the Prisoner of Azkaban (2004)',
 'Гарри Поттер и Кубок огня (фильм, 2005, 4 часть)',
 'Гарри Поттер и Принц-полукровка 2009',
 'Гарри Поттер и Кубок огня 2005',
 'Гарри Поттер и Дары Смерти: Часть I (фильм, 2010, 7 часть)',
 'Гарри Поттер и Тайная комната | Harry Potter and the Chamber of Secrets (2002)',
 'Гарри Поттер и узник Азкабана (фильм, 2004, 3 часть)',
 'Гарри Поттер и узник Азкабана (2004)',
 'Гарри Поттер и философский камень (фильм, 2001, 1 часть)',
 'Гарри Поттер и Дары смерти: Часть 2 | Harry Potter and the Deathly Hallows - Part 2 (2011)',
 'Гарри Поттер и Тайная комната (2002)',
 'Гарри Поттер и Дары смерти: Часть 1 | Harry Potter and the Deathly Hallows - Part 1 (2010)',
 'Гарри Поттер и Орден Феникса (фи

In [ ]:
video_id = "00256d60056a0251fb0e71464536bc50" # Смешарики, 2 серия
get_recs_by_i2i(rules, video_id, 30)

['Смешарики, 1 серия',
 'Смешарики, 3 серия',
 'Смешарики, 4 серия',
 'Смешарики, 5 серия',
 'Смешарики, 6 серия',
 'Смешарики, 7 серия',
 'Смешарики, 8 серия',
 'Смешарики, 9 серия',
 'Смешарики, 10 серия',
 'Смешарики, 11 серия',
 'Смешарики, 12 серия',
 'Смешарики, 13 серия',
 'Смешарики, 14 серия',
 'Смешарики, 15 серия',
 'Смешарики, 17 серия',
 'Смешарики, 16 серия',
 'Смешарики, 18 серия',
 'Смешарики, 19 серия',
 'Смешарики 2D, 6 сезон, 29 серия. Ляпус фикус хряпус',
 'Смешарики, 21 серия',
 'Смешарики, 20 серия',
 'Смешарики, 22 серия',
 'Смешарики, 23 серия',
 'Смешарики, 25 серия',
 'Тима и Тома, 3 сезон, 51 серия. Подружка',
 'Смешарики, 208 серия',
 'Дракошия, 37 серия. Волшебные бусы',
 'Смешарики, 24 серия',
 'Смешарики 2D, 6 сезон, 48 серия. Копатыч и волна',
 'Смешарики, 28 серия']

In [ ]:
video_id = "72b83e3fdefb9d5492b693a4c654ef10" # Спартак - Зенит. Обзор матча МИР РПЛ 20.08.2023
get_recs_by_i2i(rules, video_id, 30)

['Зенит - Спартак. Обзор матча Мир РПЛ 02.03.2024',
 'Динамо - Локомотив. Обзор матча Мир РПЛ 03.03.2024',
 'Спартак - Факел. Обзор матча Мир РПЛ 10.03.2024',
 'Ростов - Краснодар. Обзор матча Мир РПЛ 08.03.2024',
 'Урал - Зенит. Обзор матча Мир РПЛ 09.03.2024',
 'Спартак - ЦСКА. Обзор матча МИР РПЛ 08.10.2023',
 'Зенит - Спартак. Обзор матча FONBET Кубка России 27.11.2022',
 'Краснодар - Ростов. Обзор матча МИР РПЛ 07.10.2023',
 'Крылья Советов - ЦСКА. Обзор матча Мир РПЛ 08.03.2024',
 '«Микс тура»: эмоции 30-го тура РПЛ',
 'Факел - Спартак. Обзор матча Мир РПЛ 28.10.2023',
 'Спартак - Динамо. Обзор матча МИР РПЛ 23.09.2023',
 'Наполи - Ювентус. Обзор матча чемпионата Италии 03.03.2024',
 'Бавария - Лацио. Обзор матча Лиги чемпионов 05.03.2024',
 'Реал Сосьедад - ПСЖ. Обзор матча Лиги чемпионов 05.03.2024',
 'Пари НН - Факел. Обзор матча Мир РПЛ 03.03.2024',
 'Рубин - Спартак. Обзор матча Мир РПЛ 05.08.2023',
 'Россия - Куба. Обзор товарищеского матча 20.11.2023',
 'Зенит - Спартак - 

In [ ]:
video_id = "52adf40199ec58315a17826835b0b03a" # Comedy Club: Управляющая компания | Карибидис, Батрутдинов
get_recs_by_i2i(rules, video_id, 30)

['Comedy Club: Случай в самолёте | Карибидис, Батрутдинов',
 'Comedy Club: Такер Карлсон | Карибидис, Батрутдинов, Шальнов',
 'Comedy Club: Барды | Харламов, Батрутдинов',
 'Comedy Club: Романтический ужин | Карибидис, Кравец, Скороход, Темичева',
 'Comedy Club: 8 марта | Павел Воля',
 'Comedy Club: Утро после корпоратива | Харламов, Карибидис, Батрутдинов, Аверин, Скороход',
 'Comedy Club: Робот | Демис Карибидис, Тимур Батрутдинов, Гарик Харламов',
 'Comedy Club: Дотошная девушка | Иванов, Бутусов, Сафонов',
 'Comedy Club: Счастливый муж | Демис Карибидис, Марина Кравец',
 'Comedy Club: Новогодний корпоратив | Кравец, Карибидис, Батрутдинов',
 'Comedy Club: Витя Альварес в караоке | Карибидис, Аверин, Сорокин, Матуа, Бутусов, Торнике',
 'Камеди Клаб | Новый сезон | «Кастинг на Евровидение 2023» | Гарик Харламов, Демис Карибидис',
 'Камеди Клаб, 1 сезон, 1 выпуск',
 'Comedy Club: Cумасшедший начальник | Харламов, Батрутдинов, Бутусов, Шкуро, Шальнов',
 'Comedy Club: Устройся на работу

#### 3. Метрики качества
##### Рассчитаем метрики HitRate@10, NDCG@10, Precision@10, Recall@10, MRR@10, Serendipity@10, Novelty@10, UniqueChannels@10

In [ ]:
# Для каждого пользователя в тесте найдем его последнее просмотренное видео из трейна - делаем такой столбец
# Притягиваем к нему с помощью als рекомендации и также с помощью ассоциативных правил
# Делаем такие столбцы: viewer_id | last_video_id_train | target_list | als_recs | i2i_recs

In [ ]:
last_video_id_train = (
    df
    .sort_values("event_datetime")
    .groupby("viewer_id", as_index=False)
    .video_id
    .agg("last")
    .sort_values("viewer_id")
    .rename(columns={"video_id": "last_video_id_train"})
)
last_video_id_train

,viewer_id,last_video_id_train
0,00000666-2e86-4a04-bc49-2147c71fd754,47fdcea70b5c3002b5601dedd7aa4db7
1,000014d1-038a-46bb-8d01-82cd6837bd5c,a23e3b7aaa8d3aeb5ae4192aa3228ef4
2,00001cb558c7474b858f63b8e5484089,7476eb572501c81c8ad76648c4947416
3,0000234c-8c77-423e-a209-9c7fd2463186,89fb89610e536fd8f480df11f67f4122
4,0000279b-2013-431c-9a24-9e69f65a6cbe,4cc35c1d44b9e02755f59cc0e3ef0cd1
...,...,...
6097093,fffff48f-c0bc-439b-bcd3-8a07573715db,7bf12d9c050f9a7ef3728db5730432ae
6097094,fffff6fb-729d-4651-8cf8-98a9cb76d3f0,ff94473e331c36c3ef88fc4c557f8ee0
6097095,fffffa53-9e57-4795-955e-1d992c532e9a,0a179d0f4ce995945fbed45c736bbba1
6097096,fffffb1456d04fc791fe68f9fea880fc,e699e85dd2c7ce24a456ec327d26a806


In [ ]:
results_2 = (
    test
    .sort_values("event_datetime")
    .groupby("viewer_id", as_index=False)
    .video_id
    .agg(list)
    .sort_values("viewer_id")
    .rename(columns={"video_id": "target_video_ids"})
)
results_2

,viewer_id,target_video_ids
0,000014d1-038a-46bb-8d01-82cd6837bd5c,"[334c9d95429e789e90b76fc2524557a5, 80e6410839c..."
1,00001fd7-b624-4352-9b4d-6aa918f3f762,"[2d70b71789c56df7e1a8013c0bcb3930, 2d70b71789c..."
2,00006ccb-b15a-4943-8fcf-04f520891c87,"[dfef9acc607aec018e20c5667fe0470d, dfef9acc607..."
3,0000b335-bb03-4417-b343-91dc0b672c44,"[cdf69b8d4966a3c43fe884cd18dbddc1, c80f85e7050..."
4,0000c64c670e4db78a045bea5bef7307,"[4f2d0cf0b8f710a0c259710a4d08c755, d8c1bbe284a..."
...,...,...
1011011,ffff1180-4797-459e-95c6-cac07f18df45,"[ed2f2a992d6f59ab7d2463975b07fbde, 7b7efcfd3d8..."
1011012,ffff2cdd-8b59-4d72-af18-e58992d4d430,"[cbacbf4afe50c10380c6912884a75015, 84f061c43cc..."
1011013,ffff6435-7a50-4342-b8cf-16d473ad3f8f,"[805c350e7408f88f07d47d55bdc02a53, 805c350e740..."
1011014,ffffa74ae6ce4b109b942394f6041861,"[49d36ae0ccc2ea6ff17054296206ada5, 49d36ae0ccc..."


In [ ]:
results_3 = (
    results_2
    .merge(last_video_id_train, on="viewer_id", how="left")
    .fillna(" ")
)
results_3

,viewer_id,target_video_ids,last_video_id_train
0,000014d1-038a-46bb-8d01-82cd6837bd5c,"[334c9d95429e789e90b76fc2524557a5, 80e6410839c...",a23e3b7aaa8d3aeb5ae4192aa3228ef4
1,00001fd7-b624-4352-9b4d-6aa918f3f762,"[2d70b71789c56df7e1a8013c0bcb3930, 2d70b71789c...",
2,00006ccb-b15a-4943-8fcf-04f520891c87,"[dfef9acc607aec018e20c5667fe0470d, dfef9acc607...",
3,0000b335-bb03-4417-b343-91dc0b672c44,"[cdf69b8d4966a3c43fe884cd18dbddc1, c80f85e7050...",0dcd8a9e72221c515f7e56deb228d588
4,0000c64c670e4db78a045bea5bef7307,"[4f2d0cf0b8f710a0c259710a4d08c755, d8c1bbe284a...",
...,...,...,...
1011011,ffff1180-4797-459e-95c6-cac07f18df45,"[ed2f2a992d6f59ab7d2463975b07fbde, 7b7efcfd3d8...",
1011012,ffff2cdd-8b59-4d72-af18-e58992d4d430,"[cbacbf4afe50c10380c6912884a75015, 84f061c43cc...",d5f6b92e0344aabb9f510f1f1546e1c9
1011013,ffff6435-7a50-4342-b8cf-16d473ad3f8f,"[805c350e7408f88f07d47d55bdc02a53, 805c350e740...",
1011014,ffffa74ae6ce4b109b942394f6041861,"[49d36ae0ccc2ea6ff17054296206ada5, 49d36ae0ccc...",


In [ ]:
results_4 = results_3.sample(1_000, replace=False)

In [ ]:
video_id_to_idx = {v: k for k, v in item_ids.items()}

def get_recs_by_als(model, video_id: str, num_candidates: int) -> List[str]:
    item_index = video_id_to_idx.get(video_id, None)
    if item_index or item_index == 0:
        # skip first recommendation because it finds the same item
        recommendations = model.similar_items(
            item_index, N=num_candidates, filter_items=[item_index]
        )[0]
    else:
        return []
    return list(map(lambda x: item_ids[x], recommendations))

def get_recs_by_i2i(rules, video_id: str, num_candidates: int) -> List[str]:
    item_index = video_id_to_idx.get(video_id, None)
    if item_index or item_index == 0:
        # skip first recommendation because it finds the same item
        recommendations = rules[rules.item_id==item_index].pair.to_list()[:num_candidates]
    else:
        return []
    return list(map(lambda x: item_ids[x], recommendations))

In [ ]:
results_4["als_recs"] = results_4["last_video_id_train"].progress_apply(lambda x: get_recs_by_als(model, x, 100))

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:01<00:00, 548.24it/s]


In [ ]:
results_4["i2i_recs"] = results_4["last_video_id_train"].progress_apply(lambda x: get_recs_by_i2i(rules, x, 100))

100%|██████████| 1000/1000 [00:53<00:00, 18.54it/s]


In [ ]:
results_4["i2i_recs_len"] = results_4["i2i_recs"].progress_apply(lambda row: len(row))
results_4 = results_4.query("i2i_recs_len>0")
results_4

100%|██████████| 1000/1000 [00:00<00:00, 383216.45it/s]


,viewer_id,target_video_ids,last_video_id_train,i2i_recs,i2i_recs_len
290764,27783433-87fc-4db4-a97c-f8f8ad5ce996,"[c5a75c04add830b7f2aeef2170aac40b, a84aacac8a7...",e55b73e5687e0e88155032f6669ea8e7,"[b94cbfa1f279d29842d42c03fbb92069, 040f7db54f4...",100
220548,213684cb-e1ae-4cbf-b65a-1a8399e981e5,"[24bb85e734c7ad7bd21273df8a1c7706, ffe8eaddf7c...",d01ba26c801cd97c2e764d316f6fc1fe,"[38b3244f6d045ddf8d5035824fa28137, f10f7054b96...",100
458744,35755180,"[230891d381fd309aa915bff385b5e01c, 6c55e530e74...",dd653fd5ee95548f3a16e3748051654e,"[c0e8942a806f05f6eb1841e3777414c1, ee0ce70c6c4...",100
625363,663724744784432614,"[8c25f3a7c6a137c054e896e6381c6688, 800be356883...",a461c4d5d4732ff364ad10478594e09c,"[18422b9a2c82189d5dc508bff3b9b99d, e47f1370382...",100
340229,30717617,"[fee3e932351c1716565ae696ecb274e0, 9f567cb7cdc...",729904945a5263c4f16283275bcc59e3,"[9f1f069c78ec6d5c0b1d006ce8889b19, cdf69b8d496...",100
...,...,...,...,...,...
269608,25866111,"[ebb2f234939fb55a35454ade43207010, 698ee39117f...",040f7db54f442c7685c0d469efc7c1f3,"[66db8509d4202f73cc31a96428f7c863, e55b73e5687...",100
405417,34247356,"[787c1676105205cfd95e610bfb5ab659, 317c3853e73...",ebeed30d94681c028adeb3952d295240,"[17fecf6e04a3279927995260b4ff69a5, 654250e4a75...",100
521400,4521066708441089793,"[ffe8eaddf7c6aee7845b18d08d1bde9b, c58f502c7bb...",447a1fde27827bb3033f84df3d585ced,"[75c1d579951795b5d7732af7ddf8f0da, af9e08f0d43...",100
282301,26a24f94-2de3-41f7-8c01-4aa41e9f59c9,"[cc6d064c1373aa8231a54d588fab655d, a1ddb891998...",763d2aea1c938a41097ae87003d7bb4c,"[8eb1af34ad3fd0aab6593040b4dde7ab, f851aa51f76...",100


In [ ]:
explode_results_4 = results_4.explode("i2i_recs")
explode_results_4_target = results_4.explode("target_video_ids")

In [ ]:
def transform_sequence(seq):
    seen = set()
    result = []
    for num in seq:
        if num not in seen:
            seen.add(num)
            result.append(1)
        else:
            last_num = result[-1]
            result.append(last_num + 1)
    return result

In [ ]:
reco = pd.DataFrame(
    {
        Columns.User: explode_results_4.viewer_id.tolist(),
        Columns.Item: explode_results_4.i2i_recs.tolist(),
        Columns.Rank: transform_sequence(explode_results_4.viewer_id.tolist()),
    }
)
interactions = pd.DataFrame(
    {
        Columns.User: explode_results_4_target.viewer_id.tolist(),
        Columns.Item: explode_results_4_target.target_video_ids.tolist(),
    }
)
prev_interactions = pd.DataFrame(
    {
        Columns.User: df[df["viewer_id"].isin(explode_results_4.viewer_id.unique().tolist())].viewer_id.tolist(),
        Columns.Item: df[df["viewer_id"].isin(explode_results_4.viewer_id.unique().tolist())].video_id.tolist(),
    }
)
catalog = df[df["viewer_id"].isin(explode_results_4.viewer_id.unique().tolist())].video_id.unique().tolist()

In [ ]:
# ALS
print(f"NDCG@100: {NDCG(k=100).calc(reco, interactions)}")
print(f"MAP@100: {MAP(k=100).calc(reco, interactions)}")
print(f"MRR@100: {MRR(k=100).calc(reco, interactions)}")
print(f"Precision@100: {Precision(k=100).calc(reco, interactions)}")
print(f"Recall@100: {Recall(k=100).calc(reco, interactions)}")
print(f"Serendipity@100: {Serendipity(k=100).calc(reco, interactions, prev_interactions, catalog)}")
print(f"Novelty@100: {MeanInvUserFreq(k=100).calc(reco, prev_interactions)}")

NDCG@100: 0.01990878506378616
MAP@100: 0.08635107664638163
MRR@100: 0.13607047077273035
Precision@100: 0.010845528455284555
Recall@100: 0.21868629439937826
Serendipity@100: 0.007265557404677189
Novelty@100: 9.184493584132278


In [ ]:
# i2i
print(f"NDCG@100: {NDCG(k=100).calc(reco, interactions)}")
print(f"MAP@100: {MAP(k=100).calc(reco, interactions)}")
print(f"MRR@100: {MRR(k=100).calc(reco, interactions)}")
print(f"Precision@100: {Precision(k=100).calc(reco, interactions)}")
print(f"Recall@100: {Recall(k=100).calc(reco, interactions)}")
print(f"Serendipity@100: {Serendipity(k=100).calc(reco, interactions, prev_interactions, catalog)}")
print(f"Novelty@100: {MeanInvUserFreq(k=100).calc(reco, prev_interactions)}")

NDCG@100: 0.028393790021247287
MAP@100: 0.1188660923167066
MRR@100: 0.18389109698922687
Precision@100: 0.016179001721170396
Recall@100: 0.3772674275046741
Serendipity@100: 0.007907149205909528
Novelty@100: 7.957863915366612
